# From raw data to meaningful features
### Andreas Damianou
***For the Summer School in Data Science and Machine Learning, 
Makerere University, Uganda, 27 June 2016***


## INSTRUCTIONS - README

This notebook will guide you through some basic theory of working with features. Go through it sequentially, run all the code step by step. Some code cells are already complete and you just have to run them. Some other cells (denoted with **TODO**) require you to fill in the code.

If you get stuck in one of the questions, don't lose too much time, ask a demonstrator for help or move on to the next question.

When the code is already there, make sure you understand what it's doing, don't just run it and continue without understanding it!

**!!! So, look for parts with **TODO** and make sure you fill in the correct answer to the questions.***

First we import the plotting and numerical libraries.

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import numpy as np


## Start with toy data for classification

We'll consider the task of extracting features for some data that are meant to be used for classification.

So, before anything, let's start revising the code for K-Nearest Neighbours algorithm.

In [ ]:
#  CODE for NN:
def knn(Y_training, labels_training, y_test, k):
    from scipy.spatial import distance
    from operator import itemgetter
    
    # distance_test_to_train[i] says what's the distance of the i-the training point to the y_test
    distance_test_to_train = []
    for i in range(Y_training.shape[0]):
        distance_test_to_train.append(distance.euclidean(Y_training[i,:], y_test))
    # The following list sorts the index of the distance list
    sorted_dist_indexes = np.argsort(distance_test_to_train)
    
    # The indexes we want to return are only the top k 
    indexes = sorted_dist_indexes[0:k]
    
    # Test label is the najority of neighbours' labels
    label_test = max(set(labels_training[indexes]), key=labels_training[indexes].tolist().count)
    
    return label_test, indexes, distance_test_to_train



After we implement a new functionality in our code, it is very important to test it and verify that it's doing what we think it's doing. The simplest way to do that, is to construct some fake data and run the newly constructed function of them. Since it's our own fake data, we know the correct answer that the function should give, so we can compare that to what the code returns.

We'll test the knn function. For this we'll need to create some training data called for example: `my_Y_training`, associated with some labels: `my_labels_training`. Then, create some test instances `my_y_test` and test the code by calling:

`knn(my_Y_training, my_labels_training, my_y_test, 2)`. 

The last argument is the number of neighbours, here selecfted to 2, but try different values too.

In [ ]:
# Create toy data
my_Y_training = np.array([[0,1],[2,2],[10,15],[2,1],[2,15],[0,1],[8,14],[5,5],[6,2],[7,6],[4,10],[4,6]])
my_labels_training = np.array([0,0,1,0,1,0,1,0,0,1,1,0])
my_Y_test = np.array([[0,2], [9,12], [8,4],[6.5,6]])
print(my_Y_training)

# Create a list whose each element is the colour of each training point's class.
# It's read or green, depending if the label is 0 or 1.
colours=[('r' if i==0 else 'g') for i in my_labels_training]

# Do a plot of the points. Since the points are two-dimensional (two columns), 
# the scatter plot is going to be a 2D plot. 
plt.scatter(my_Y_training[:,0],my_Y_training[:,1],marker='o',s=78,c=colours)



**TODO Q1:** Try different values for the number k below.

In [ ]:
# TODO: try other values too
k=2

In [ ]:
# Call the knn code for each test instance and get back the label. All test inferred labels are collecfted into a list.
labels_test = [knn(my_Y_training, my_labels_training, my_Y_test[i:i+1,:],k)[0] for i in range(my_Y_test.shape[0])]
# Repeat the scatter plot of above, just to have all points (test and training) together
plt.scatter(my_Y_training[:,0],my_Y_training[:,1],marker='o',s=78,c=colours)

colours=[('r' if i==0 else 'g') for i in labels_test]
# Create a scatter plot where test data are denoted by 'x' and have clour green or red, according to what label they were assigned.
plt.scatter(my_Y_test[:,0],my_Y_test[:,1],marker='x',s=78,c=colours)




**TODO Q2**: What do you observe as you set k=1 or k=3? 

** Your answer goes here **


*** TODO Q3***: Now you can do the same test as above but with 3 dimensional data. Fill in the code with toy data in three dimensions.

In [ ]:
# Same as above but in 3 dimensions.
from mpl_toolkits.mplot3d import Axes3D

# TODO-----
my_Y_training = 
my_labels_training = 
my_Y_test = 
#-------

print(my_Y_training)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
colours=[('r' if i==0 else 'g') for i in my_labels_training]
ax.scatter(my_Y_training[:,0],my_Y_training[:,1],my_Y_training[:,2],marker='o',s=78,c=colours)

k=2 # TODO: Also try with 1
labels_test = [knn(my_Y_training, my_labels_training, my_Y_test[i:i+1,:],k)[0] for i in range(my_Y_test.shape[0])]
print(labels_test)
colours=[('r' if i==0 else 'g') for i in labels_test]
ax.scatter(my_Y_test[:,0],my_Y_test[:,1],my_Y_test[:,2],marker='x',s=78,c=colours)


Now we'll test feature selection. For this, we'll need a dataset with many features per instance (i.e. `Y` will have many columns, here we'll try 5). We'll investigate how the results change as we consider different subsets of features to use.

First, we'll create some toy data as before. 

In [ ]:
# Data
my_Y_training = np.array([[0,1,0,2,6],[2,2,0,2,1],[10,8,0.0001,11,1],[2,1,0,2,12],[2,15,0.001,9,1],[0,1,0,8,11],[8,14,0.0001,10,6]])
my_labels_training = np.array([0,0,1,0,1,0,1])
my_Y_test = np.array([[0,2,0,3,6], [9,12,0,8,6], [8,4,0,8,6]])

Now we need to decide:
1. How many features to use?
2. Which features?

There are many methods to do that. Here we'll explore some very basic ones.

The simplest and more intuitive, is to select features by interacting with our data. Remember, the feature-selection procedure must be carried out by considering what **task** we have in mind (in our case, classification). Therefore, we'll first just plot our data alongside the labels

In [ ]:
fig, axes = plt.subplots(1,2,sharey=True)

axes[0].matshow(my_Y_training,aspect='auto')
axes[1].matshow(my_labels_training[:,None],aspect='auto')
axes[0].set_title('Y data')
axes[1].set_title('Labels')


**TODO Q4**

Just by looking at the data, answer:
1. Which features look more descriptive with respect to labels?
2. Which seem less descriptive?
3. In general, could it be the case that we can increase descriptiveness by considering set (combination) of features rather than just picking one? Why?

** Your answer goes here **
1. 
2. 
3. 


When we have a truly large number of features to consider, it'll be diffult to rely on visualisation. However, we can still get a data summary by writing some code.

Firstly, from the above plot we see that feature in column 2 is almost constant. This can be programmatically observed by computing the variance across features:

In [ ]:
print(my_Y_training.var(0))

Of course, we can't be sure that the infinitesimal variance around this feature is indeed neglectable. If we were doing unsupervised learning, i.e. we didn't have the label set with us, we'd probably reject this feature as it seems useless.

However, here we have the labels, so we can verify that this feature is "useless" by checking if it follows a pattern similar to the labels. Because it's hard to verify this from the above matrix, we can magnify the differences in this feature by re-normalizing it. This motivates the idea of re-normalizing all features to compare them to the label vector. Re-normalization means that we'll make all features be between 0 and 1.

In [ ]:
my_Y_training_norm = np.zeros((my_Y_training.shape[0],my_Y_training.shape[1]))
for j in range(my_Y_training.shape[1]):
    tmp = my_Y_training[:,j]
    my_Y_training_norm[:,j] = (tmp - tmp.min(0)) / tmp.ptp(0)
    

Plot the normalized data and the label vector next to them.

In [ ]:
fig, axes = plt.subplots(1,2,sharey=True)

axes[0].matshow(my_Y_training_norm,aspect='auto')
axes[1].matshow(my_labels_training[:,None],aspect='auto')
axes[0].set_title('Y data')
axes[1].set_title('Labels')


Now we see that feature 2 is actually not that useless! It is slightly larger for data (rows of Y) that correspond to label 1.

***TODO Q5***

Let's automate the above discussion. We'll again follow a very simple approach which returns the distance of each normalized feature vector to the label vector, the intuition being that if the distance is small, then that might be a good feature.

*Hint*: check the knn code to see how the distances are taken.

In [ ]:
# TODO: Your code here



Although we've discussed that usually combination of features (rather than just selecting one feature) is expected to give the best results, most of the code we wrote above focuses on looking at features individually.

Finding combination of features is more difficult, as the problem of considering all possible combinations makes the search space larger. We won't explore state-of-the-art methods here, but instead we'll just get a feeling as to what could be a good feature combination for our data by trying several subsets of features and measuring the classification accuracy on the test set (which in this case is rather a validation set).

***TODO Q6:*** 
1. Make the data so that you use only 1 feature.
2. Plot the 1-dimesional data with different colors according to the labels and then for the test point y*
3. find the k-nns and infer its label. What k will you use here? Why?
4. Print the distances. 
5. Do this for different selection of feature. Which is best?

*** TODO Q7:***  
6. Now select two features (any combination you want) for your data and repeat as above.
7. Repeat, this time by selecting 3 features.

In [ ]:
# YOUR CODE GOES HERE

*** TODO Q8:***
1. Which combination of features yields better results in terms of classification accuracy? 


** YOU ANSWER GOES HERE **

** TODO: Q9:**  What happens to the distances as you add features?

** Your answer here**

## Fun with text data

Let's move away from toy data. We'll now look at more interesting data, namely text data. 
We'll create some fake texts. Each text is associated with a label which is 1 if the text is spam email, otherwise it has label 0.

In [ ]:
text = dict()
# A vector so that labels[i] is 1 if the text is spam otherwise 0.
labels = [1,1,0,0,1,1,0]

text['0'] = ['this','email','is','a','spam','email']
text['1'] = ['spam','spam','spamity','spam']
text['2'] = ['this','email','is','about','python','and','python','example']
text['3'] = ['python','is','good','for','you','python','good']
text['4'] = ['you','should','not','learn','python']
text['5'] = ['this','email','is','bad','email','and','you','should','not']
text['6'] = ['this','is','a','good','email','good','as','it','can','get']

num_texts = len(text.keys())

# Concatenate all texts into a single big text. This will be helpful later.
texts = []
for tt in text.keys():
    texts += text[tt]
texts

In [ ]:
# Find the unique words in all texts by looking at the unique elements of the big concatenated text.
uniques = np.unique(texts)
print(uniques)


We now have a list of all the unique words appearing in our text collection.

Many of the words are very simple, like 'a', 'I' etc. We'd like to get rid of them. Why? Because we assume that they're so simple and versatile that they can't offer much statistical insight into the nature of the text.

This process is called stemming.

In [ ]:
# Define the dictionary of words that you wish to ignore from your texts
stemming_list = ['a','the','for','and','is','I','you','he','she','it','they']
# Helper variable, just get the unique words and transform them to list
uniques_list = uniques.tolist()
for i in range(len(stemming_list)): 
    try:
        # Remove the i-th word if it is in the stemming list
        uniques_list.remove(stemming_list[i])
    except ValueError:
        pass
# Now turn the list of uniques (stemmed) back into an array
uniques = np.array(uniques_list)
print(uniques)

In [ ]:
# Sanity check: Print the first text
tmp = text['0']
print(tmp)


*** TODO Q10 ***:

Now the important bit. We have texts as collection of words, cleaned and stemmed.

How do we turn this into useful features?

Think about your answer before scrolling down (don't cheat!!)
Write your answer below.


*** Your answer goes here ***

*** SCROLL DOWN, there's more!!! ........ ***

*** SCROLL DOWN, there's more!!! ........ ***

## Features from Text: Creating a histogram - Bag of Words

Here's one possible answer to Q10: We'll create features from text, by  making a histogram in a manner which is called a **Bag of Words**.

A Bag of Words means that you'll convert each text into a **fixed sized** vector $[w_1, w_2, ... w_d]$, where $d$ is the total number of different words you may encounter, and $w_i$ tells us how many times this particular word appeared in the text. 

In [ ]:
text_hist = np.zeros((num_texts,len(uniques)))
for i in range(num_texts):
    for j in range(len(uniques)):
        for k in range(len(text[str(i)])):
            if text[str(i)][k] == uniques[j]:
                text_hist[i,j]+=1
print(text_hist)

*** TODO Q11 ***: 

1. Explain in your own words what is the Bag of Words representation.
2. What are the limitations of this representation? What are the advantages?

*** Your answer goes here ***

Some features appear only once, e.g. those in the first four columns. 
We can't do much statistical inference if these features appear only in one instance. Hence, we'll remove them.

In [ ]:
inds = np.where(text_hist.sum(0) > 1)[0]
text_hist = text_hist[:,inds]
uniques = uniques[inds]

Now we'll plot the (remaining) features alongside the labels.

In [ ]:
fig, axes = plt.subplots(1,2,sharey=True)
axes[0].matshow(text_hist, interpolation='none',aspect='auto')
axes[0].set_xticklabels(['']+uniques.tolist(),rotation='vertical');
axes[0].set_title('Data Features',y=1.2)
axes[1].matshow(np.array(labels)[:,None], aspect='auto')
axes[1].set_title('Data Labels',y=1.2)

# Fun with images

### Bonus! Compute SURF features (requires openCV)

In [ ]:
# Select some images and compute SURF. See that there's a parameter there. What is a reasonable one?

source: http://docs.opencv.org/

In [ ]:
import cv2
img = cv2.imread('/home/andreas/Andreas.jpg',0)

In [ ]:
# Create the surf object. The parameter is the threshold, roughly controlling the sensitivity to edges.
surf = cv2.SURF(300)
# Get the keypoints and descriptors for the image
kp, des = surf.detectAndCompute(img,None)
# Check how many keypoints were found
print(len(kp))

In [ ]:
# Draw the keypoints on top of the image
img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img2),plt.show()

*** TODO Q12 ***
1. Play with the code above. Load any picture you want, and compute the SURF features. Adjust the threshold parameter to get more or less features.
2. If you have more than one pictures, how can you extract consistent features across all pictures? *Hint*: Remember how we did that for text data. Here you might need to do an additional clustering step after you extract the "words" (in this case instead of words you have SURF descriptors), and now the histogram will say how many descriptors of each cluster we have.


# Extra:

If you've reached here and you still have time, try doing these exercises:
1. Go back to the text processing section, and fit a classifier. See how the model performs and how you can make it better.
2. Go back to the toy classification Q8 and argue about which feature combination is best by this time splitting the data into training and validation set to automatically select a good combination (e.g. search over a predifined selection of feature combinations).

## Fun with Images - This part is challenging, I leave it here if you want to have fun on your own time!

In [ ]:
# If PIL is not installed, then only png can be read
img = np.array(imread('PATH TO YOUR IMG')) 

# Lazy conversion from RGB to grayscale
img = img[:,:,0]
plt.imshow(img,cmap=plt.get_cmap('gray'))


In [ ]:
import glob

imagePath = '' # HERE YOU PUT the path of the images folder, where you have your images stored. They have to be of consistent dimensions
extensions = 'jpg' # Change if your images is something else

listing = glob.glob(imagePath + '*.' + extensions)
(height,width) = np.array(imread(listing[0],True))[:,:,0].shape
print(height,width)
images = np.array([imread(i,True)[:,:,0].flatten() for i in listing])

plt.imshow(images[0,:].reshape(height,width), cmap=plt.get_cmap('gray'))



In [ ]:
# Compute and plot the mean face
mu = np.mean(images,0)
plt.imshow(mu.reshape(height,width),cmap=plt.get_cmap('gray'))


In [ ]:
# Center data
images_cent = images-mu

U,S,V = linalg.svd(images_cent.transpose(), full_matrices=False)

In [ ]:
eigenfaces = U.T

print(images.shape)
print(eigenfaces.shape)

# Each row of U is an eigen-face
plt.imshow(eigenfaces[0,:].reshape(height,width),cmap=plt.get_cmap('gray'))

In [ ]:
# Reconstruction
weights = np.dot(images_cent, eigenfaces.T)
print(weights.shape, eigenfaces.shape, images.shape)
j=0

img_j_reconstr = mu + np.dot(weights[j,:],eigenfaces)

plt.imshow(img_j_reconstr.reshape(height,width),cmap=plt.get_cmap('gray'))

In [ ]:
# Reconstruction by using fewer eigenfaces 
k=3
img_j_reconstr_k = mu + np.dot(weights[j,0:k], eigenfaces[0:k])
plt.imshow(img_j_reconstr_k.reshape(height,width),cmap=plt.get_cmap('gray'))

In [ ]:
# Credits: wellecks.wordpreess.com/tag/eigenfaces

In [ ]:
# Also try to do feature selection: Select only an area around the eyes.